In [259]:
# import dependencies
import numpy as np
import scipy as sp
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib as jl
import datetime as dt
import requests

1. go to that github code and just switch it for all my etfs
2. change to paper trading api

In [260]:
import os
from dotenv import load_dotenv
load_dotenv()

TOKEN = os.getenv('TOKEN')

In [261]:
# # create watchlist
# response = requests.post('https://api.tradier.com/v1/watchlists',
#     data={'name': 'CNBC_INDEX_LIST', 'symbols': 'SPY,QQQ,IWM,DIA,VTI,MDY,DBC,FEZ,OEF,IWF,IWD,PFF,VOO,IJH,IWO,IWN,ACWI,IEMG'},
#     headers={'Authorization': TOKEN, 'Accept': 'application/json'}
# )
# json_response = response.json()

In [262]:
# pull watchlist

response = requests.get('https://api.tradier.com/v1/watchlists/CNBC_INDEX_LIST',
    params={},
    headers={'Authorization': TOKEN, 'Accept': 'application/json'}
)
json_response = response.json()
print(response.status_code)

200


In [263]:
tickers_df = pd.DataFrame.from_records(json_response['watchlist']['items']['item'])
tickers = list(pd.read_csv('data/etf_list.csv').Symbol)

In [264]:
# pull historical data

# reorganize into oop
d = {}

for i in range(len(tickers)): 

    response = requests.get('https://api.tradier.com/v1/markets/history',
        params={'symbol': tickers[i], 'interval': 'daily', 'start': '2019-01-01', 'end': '2023-01-01'},
        headers={'Authorization': TOKEN, 'Accept': 'application/json'}
    )
    json_response = response.json()
    print("Response Status Code:", response.status_code)
    
    for symbol in tickers:
        d[symbol] = pd.DataFrame.from_records(json_response['history']['day'], index='date')

Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Status Code: 200
Response Sta

In [265]:
response = requests.get('https://api.tradier.com/beta/markets/fundamentals/financials',
    params={'symbols': 'MSFT'},
    headers={'Authorization': TOKEN, 'Accept': 'application/json'}
)
json_response = response.json()
print(response.status_code)
#print(json_response)

200


In [274]:
#dict1 = json_response[0]['results'][0]['tables']

In [275]:
#dict1['financial_statements_restate']['income_statement'][0]['period_12m']

In [273]:
#d[symbol] = pd.DataFrame.from_records(json_response['history']['day'], index='date')

In [269]:
# add multi-index with symbols and turn into dask df
df = pd.concat(d.values(), axis=1, keys=d.keys())

In [270]:
dask_df = dd.from_pandas(df, npartitions=6)

In [272]:
dask_df.head()

SPY                                    IVV                  \
             open     high    low  close   volume   open     high    low   
date                                                                       
2019-01-02  63.50  63.5100  63.43  63.49  1198519  63.50  63.5100  63.43   
2019-01-03  63.54  63.9300  63.54  63.91   490042  63.54  63.9300  63.54   
2019-01-04  63.60  63.6263  63.49  63.53  1179975  63.60  63.6263  63.49   
2019-01-07  63.56  63.6000  63.36  63.36   577236  63.56  63.6000  63.36   
2019-01-08  63.31  63.3750  63.26  63.26   310521  63.31  63.3750  63.26   

                            ...    XLI                                   VGIT  \
            close   volume  ...   open     high    low  close   volume   open   
date                        ...                                                 
2019-01-02  63.49  1198519  ...  63.50  63.5100  63.43  63.49  1198519  63.50   
2019-01-03  63.91   490042  ...  63.54  63.9300  63.54  63.91   490042  63.54   
2019-01-04  63.53  1179975  ...  63.60  63.6263  63.49  63.53  1179975  63.60   
2019-01-07  63.36   577236  ...  63.56  63.6000  63.36  63.36   577236  63.56   
2019-01-08  63.26   310521  ...  63.31  63.3750  63.26  63.26   310521  63.31   

                                            
               high    low  close   volume  
date                                        
2019-01-02  63.5100  63.43  63.49  1198519  
2019-01-03  63.9300  63.54  63.91   490042  
2019-01-04  63.6263  63.49  63.53  1179975  
2019-01-07  63.6000  63.36  63.36   577236  
2019-01-08  63.3750  63.26  63.26   310521  

[5 rows x 500 columns]

In [271]:
print(len(dask_df), '\n', len(dask_df.columns))

1008 
 500
